Forms k clusters (k found using inertia plots, choosing the elbow point), and takes average of Semantic Orientation scores of tweets in the cluster to classify tweets (KMeans)

In [2]:
import pandas as pd 
from scipy import spatial 
import gensim.models
import json
import numpy as np 
from sklearn.cluster import KMeans

C:\Users\Aman Garg\AppData\Roaming\Python\Python27\site-packages\gensim-2.1.0-py2.7-win-amd64.egg\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
No handlers could be found for logger "gensim.models.doc2vec"


In [3]:
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

In [4]:
ff=open('C:\Users\Aman Garg\Desktop\Project\Codes\phrases_final_updated.json','r') #Opening json file containing phrases extracted from tweets
ph=json.load(ff)
so={}
solist=[]
phr={}

In [5]:
model = gensim.models.KeyedVectors.load_word2vec_format('C:\Users\Aman Garg\Documents\GoogleNewsDataset\GoogleNews-vectors-negative300.bin', binary=True)


In [6]:
not_found_counter = 0 # Counter for phrases not found in word2vec
tweet_vec_list = []
for j in range(8000):
	k=unicode(j)
	phr[k]=ph[k] #phr[k] - list of phrases for tweet number k
	phrase_vec_list = []
	for i in phr[k]:
		#print i
		phrase_words = i.split()
		#print phrase_words
		try:
			word_sum = (model[phrase_words[0]]+ model[phrase_words[1]])/2
			phrase_vec_list.append(word_sum)
		except:
			not_found_counter = not_found_counter + 1
			pass
	try:
		tweet_vec = sum(phrase_vec_list)/len(phrase_vec_list)
	except:
		tweet_vec = [0]*300
	tweet_vec_list.append(tweet_vec)
	#print tweet_vec

In [7]:
samples = np.array(tweet_vec_list)
new_samples = samples[:]
samples = samples[:]

ks = range(1,40)
Inertias = []
for k in ks:
    kmeans = KMeans(n_clusters = k)
    labels = kmeans.fit(samples)
    Inertias.append(kmeans.inertia_)
# Plot ks vs inertias
plt.plot(ks, Inertias, '-o')
plt.xlabel('number of clusters, k')
plt.ylabel('inertia')
plt.xticks(ks)
plt.show()


Choose optimal k (elbow point)

After Standardization

In [48]:
# Put opt k
k_opt = 6
kmeans = KMeans(n_clusters = k_opt)

In [49]:
scaler = StandardScaler()
pipeline = make_pipeline(scaler,kmeans)
pipeline.fit(samples)
y_pred = pipeline.predict(new_samples)

In [50]:
print len(y_pred)

8000


In [51]:
df2 = pd.read_csv('C:\Users\Aman Garg\Desktop\Project\Codes\word2vec_accuracy.csv')

In [52]:
y_test_so =df2['similarity'].values
y_test = y_test_so[:]
#print len(y_test)
#print len(y_pred)
cluster_so = []
for i in range(k_opt):    
    print "cluster "+str(i)+ " count:"+str(sum(y_pred == i))
    so = sum(y_test[y_pred == i])
    print "so:" + str(so/sum(y_pred == i))
    cluster_so.append(so/sum(y_pred == i)) #Appending the average so


cluster 0 count:536
so:0.0541972662985
cluster 1 count:1294
so:0.0673922701677
cluster 2 count:1293
so:0.0381925788213
cluster 3 count:2293
so:0.0712066047065
cluster 4 count:1002
so:0.180413167374
cluster 5 count:1582
so:-2.5730306574e-05


In [32]:
cluster_label = []
for i in cluster_so:
    if i < 0:
        cluster_label.append(0)
    elif i>0:
        cluster_label.append(1)
    else:
        cluster_label.append(-1)
print (cluster_label)
for index,i in enumerate(y_pred):
    y_pred[index] = cluster_label[y_pred[index]]
    

[0, 1, 1, 1, 1, 0, 1, 1, 1, 1, -1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1]


In [175]:
count = 0
pos_pred_count = 0
neg_pred_count = 0
for index,value in enumerate(y_pred):
    if y_test[index] == 'positive' and value == 1:
        count = count +1
    elif y_test[index] == 'negative' and value == 0:
        count = count + 1
    if value == 1:
        pos_pred_count = pos_pred_count + 1
    else:
        neg_pred_count = neg_pred_count + 1
print 'Accuracy: %d/8000' %count
print 'Positive predictions :' + str(pos_pred_count)
print 'Negative predictions :' + str(neg_pred_count)
# for k = 100, with training -> 6000 and test -> 2000 : Accuracy 1510/2000
# for k = 20, with training -> 6000 and test -> 2000 : Accuracy 1478/2000
# for k = 2, with training -> 6000 and test -> 2000 : Accuracy 1478/2000
# for k = 40, with training -> 6000 and test -> 2000 : Accuracy 1499/2000

#for k = 40, with training and testing 8000 (same set) : Accuracy 5820/8000
#for k = 2, with training and testing of 8000 (same set) : Accuracy 5649/8000
#for k = 20, with training and testing of 8000 (same set) : Accuracy 5813/8000

Accuracy: 5813/8000
Positive predictions :7416
Negative predictions :584
